In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import FeatureUnion

In [2]:
def printgr(text):
    print(colored.stylize(text, colored.fg('green')))

In [3]:
cd /export/usuarios_ml4ds/lbartolome/gFedNTM

/export/usuarios_ml4ds/lbartolome/gFedNTM


In [4]:
from src.models.base.pytorchavitm.datasets.bow_dataset import BOWDataset

In [5]:
path_corpus = Path("/export/usuarios_ml4ds/lbartolome/gFedNTM/aux/corpus.parquet")

In [6]:
df = pd.read_parquet(path_corpus)

In [7]:
corpus_node0 = df[df['fieldsOfStudy'] == "sociology"]
corpus_node1 = df[df['fieldsOfStudy'] == "philosophy"]

In [10]:
# Preparo los documentos en el formato de entrada para hacer el fit del CountVectorizer
lemmas_node0 = corpus_node0[["bow_text"]].values.tolist()[0:10]
lemmas_node0 = [doc[0].split() for doc in lemmas_node0]
lemmas_node0 = [el for el in lemmas_node0]
lemmas_node0 = [" ".join(lemmas_node0[i]) for i in np.arange(len(lemmas_node0))]


lemmas_node1 = corpus_node1[["bow_text"]].values.tolist()[0:10]
lemmas_node1 = [doc[0].split() for doc in lemmas_node1]
lemmas_node1 = [el for el in lemmas_node1]
lemmas_node1 = [" ".join(lemmas_node1[i]) for i in np.arange(len(lemmas_node1))]

In [11]:
# Creo los CountVectorizer de cada nodo
cv0 = CountVectorizer(input='content', lowercase=True,
                      stop_words='english', binary=False)
train_bow0 = cv0.fit_transform(lemmas_node0).toarray()
idx2token0 = cv0.get_feature_names_out()
input_size0 = len(idx2token0)
print(input_size0)

cv1 = CountVectorizer(input='content', lowercase=True,
                      stop_words='english', binary=False)
train_bow1 = cv1.fit_transform(lemmas_node1).toarray()
idx2token1 = cv1.get_feature_names_out()
input_size1 = len(idx2token1)
print(input_size1)

471
392


In [12]:
cv0.vocabulary_

{'social': 403,
 'present': 330,
 'problem': 334,
 'economic': 148,
 'propose': 342,
 'change': 65,
 'article': 33,
 'country': 103,
 'issue': 235,
 'way': 458,
 'support': 432,
 'apply': 24,
 'international': 230,
 'right': 376,
 'society': 405,
 'program': 340,
 'energy': 154,
 'address': 9,
 'concern': 85,
 'help': 205,
 'institution': 226,
 'reform': 356,
 'response': 371,
 'principle': 332,
 'explain': 175,
 'family': 182,
 'environmental': 158,
 'movement': 279,
 'century': 63,
 'basic': 43,
 'american': 20,
 'party': 304,
 'worker': 466,
 'turn': 445,
 'rural': 379,
 'land': 241,
 'start': 417,
 'justice': 238,
 'poor': 318,
 'democratic': 120,
 'farmer': 184,
 'weak': 459,
 'sound': 408,
 'enter': 155,
 'farm': 183,
 'migrant': 271,
 'cooperative': 98,
 'ecological': 147,
 'campaign': 51,
 'middle': 270,
 'interested': 228,
 'join': 237,
 'capitalism': 54,
 'affair': 13,
 'director': 134,
 'realm': 352,
 'socialism': 404,
 'harmony': 203,
 'catholic': 59,
 '1950': 2,
 '1930': 0

In [28]:
# Creo el FeatureUnion
vocabs = [(("CV0", CountVectorizer(vocabulary=cv0.vocabulary_))),
          (("CV1", CountVectorizer(vocabulary=cv1.vocabulary_)))]

union = FeatureUnion(vocabs)
idx2token = union.get_feature_names_out()
print(len(idx2token)) # Está creando un vocabulario de tamaño la suma de los individuales, 
                      # pero no está teniendo en cuenta palabras que aparezcan en ambos

6595


In [38]:
# Hago el fit de los documentos del primer nodo
# Esta es la información con la que tendría que crear el BoWDataset en el nodo 0 (le paso el train_bow y el id2token)
train_bow = union.transform(lemmas_node0).toarray()
idx2token = union.get_feature_names_out()
id2token = {k: v for k, v in zip(range(0, len(idx2token)), idx2token)}

print(len(idx2token))
id2token

6595


{0: 'CV0__00',
 1: 'CV0__000',
 2: 'CV0__10',
 3: 'CV0__104',
 4: 'CV0__11',
 5: 'CV0__12',
 6: 'CV0__120',
 7: 'CV0__13',
 8: 'CV0__137',
 9: 'CV0__138',
 10: 'CV0__142',
 11: 'CV0__15',
 12: 'CV0__16',
 13: 'CV0__163',
 14: 'CV0__164',
 15: 'CV0__1640',
 16: 'CV0__1650',
 17: 'CV0__1660',
 18: 'CV0__17',
 19: 'CV0__1730',
 20: 'CV0__1791',
 21: 'CV0__18',
 22: 'CV0__1827',
 23: 'CV0__1848',
 24: 'CV0__1851',
 25: 'CV0__1852',
 26: 'CV0__1858',
 27: 'CV0__1868',
 28: 'CV0__1888',
 29: 'CV0__1890',
 30: 'CV0__19',
 31: 'CV0__1901',
 32: 'CV0__1906',
 33: 'CV0__1911',
 34: 'CV0__1918',
 35: 'CV0__1927',
 36: 'CV0__1937',
 37: 'CV0__1941',
 38: 'CV0__1943',
 39: 'CV0__1945',
 40: 'CV0__1947',
 41: 'CV0__1949',
 42: 'CV0__1950',
 43: 'CV0__1951',
 44: 'CV0__1952',
 45: 'CV0__1953',
 46: 'CV0__1954',
 47: 'CV0__1955',
 48: 'CV0__1956',
 49: 'CV0__1957',
 50: 'CV0__1958',
 51: 'CV0__1959',
 52: 'CV0__196',
 53: 'CV0__1960',
 54: 'CV0__1962',
 55: 'CV0__1963',
 56: 'CV0__1964',
 57: 'CV0__19